In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "acl"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-3-at-2023-03-06-16-21-857f69e6.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_acl_export_big_v2_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 36658,
 'annotations': [{'id': 5501,
   'completed_by': 1,
   'result': [{'value': {'start': 54,
      'end': 65,
      'text': 'reklaamitud',
      'labels': ['acl_phrases']},
     'id': 'GhMfA0ffws',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['free']},
     'id': '8Hmj8cHPiY',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-02-23T12:36:20.106673Z',
   'updated_at': '2023-02-23T12:36:20.106728Z',
   'lead_time': 19.036,
   'prediction': {'id': 36658,
    'model_version': 'undefined',
    'created_ago': '0\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 54,
       'end': 65,
       'text': ['reklaamitud'],
       'labels': ['acl_phrases']}}],
    'score': None,
    'cluster': None,
    'n

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

39393


In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,36658,Train/aja_ee200110_osa_3_ud211.conllu,"Piirkonna detailplaneeringust selgub , et ka k...",54,65,reklaamitud,free,"Piirkonna detailplaneeringust selgub , et ka k...",None,None,None
1,36659,Train/aja_pm20001004_osa_3_ud211.conllu,Viljandi maavalitsuse haridus- ja kultuuriosak...,157,180,õppetööst eemal olevale,bound,Viljandi maavalitsuse haridus- ja kultuuriosak...,96.3,96.3,100.0
2,36660,Train/aja_pm19980926e_ud211.conllu,Uskumatult suus sulav lõpptulemus paneb õuna e...,0,21,Uskumatult suus sulav,free,lõpptulemus paneb õuna ees mütsi kergitama ja ...,93.8,93.8,100.0
3,36661,Train/aja_pm19980926e_ud211.conllu,Uskumatult suus sulav lõpptulemus paneb õuna e...,87,127,muuta iseenesest lihtne kook taevalikuks,bound,Uskumatult suus sulav lõpptulemus paneb õuna e...,92.9,92.9,100.0
4,36662,Train/aja_ee199920_osa_1_ud211.conllu,"Selle asemel , et avaldada vabandus oma töötaj...",13,74,", et avaldada vabandus oma töötaja ebaõnnestun...",redundant comma,"Selle asemel , asub toimetus Tanel Padarit eda...",82.8,93.1,89.7
...,...,...,...,...,...,...,...,...,...,...,...
2749,39413,Train/tea_AA_05_6_ud211.conllu,Kui protsessorüksus sooritab ainuvallatava ope...,29,42,ainuvallatava,bound,Kui protsessorüksus sooritab operandi laadimis...,90.0,90.0,95.0
2750,39414,Train/tea_AA_05_6_ud211.conllu,Kui protsessorüksus sooritab ainuvallatava ope...,141,159,süsteemi ühendatud,free,Kui protsessorüksus sooritab ainuvallatava ope...,94.7,94.7,100.0
2751,39415,Train/aja_luup200106_osa_5_ud211.conllu,Kapitalistlikus ühiskonnas mängivad aga just s...,76,170,", kes hierarhilisel karjääriredelil kui kaugel...",free,Kapitalistlikus ühiskonnas mängivad aga just s...,100.0,100.0,100.0
2752,39416,Dev/aja_ee199920_osa4_ud211.conllu,"( Teoretiseeritud on muidugi ka , et meediarea...",112,135,meedia poolt vahendatud,bound,"( Teoretiseeritud on muidugi ka , et meediarea...",94.4,94.4,100.0


In [11]:
df["type"].unique()

array(['free', 'bound', 'redundant comma', 'unnatural', 'dubious',
       'other redundant punctuation'], dtype=object)

In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,36658,Train/aja_ee200110_osa_3_ud211.conllu,"Piirkonna detailplaneeringust selgub , et ka k...",54,65,reklaamitud,free,"Piirkonna detailplaneeringust selgub , et ka k...",None,None,None
2,36660,Train/aja_pm19980926e_ud211.conllu,Uskumatult suus sulav lõpptulemus paneb õuna e...,0,21,Uskumatult suus sulav,free,lõpptulemus paneb õuna ees mütsi kergitama ja ...,93.8,93.8,100.0
5,36663,Train/aja_ee199920_osa_1_ud211.conllu,"Selle asemel , et avaldada vabandus oma töötaj...",48,61,ebaõnnestunud,free,"Selle asemel , et avaldada vabandus oma töötaj...",83.8,91.9,91.9
8,36666,Train/aja_pm19980926e_ud211.conllu,Möödunud aastal viiekümnenda juubeli tähistami...,0,62,Möödunud aastal viiekümnenda juubeli tähistami...,free,festivalist võttis osa 1200 zhongleerijat .,83.3,100.0,83.3
9,36667,Train/arborest_ud211.conllu,Sõmera toetab plaani viia umbes viiendik telek...,21,86,viia umbes viiendik telekomi aktsiatest börsil...,free,Sõmera toetab plaani .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2745,39409,Dev/aja_epl20070812_ud211.conllu,Umbes 6000 pealtvaataja silme all Pärnumaal Au...,100,113,starti tulnud,free,Umbes 6000 pealtvaataja silme all Pärnumaal Au...,91.3,91.3,95.7
2746,39410,Train/aja_luup200202_osa_2_ud211.conllu,Tsaariajal ja nõukogude perioodil mahavaikitud...,0,46,Tsaariajal ja nõukogude perioodil mahavaikitud,free,hõimu- ja klannisuhted elustusid NL-i lagunede...,100.0,100.0,100.0
2747,39411,Train/aja_ml200247_osa_2_ud211.conllu,Kuna riik näitab toetustega üles huvi piimatoo...,38,61,piimatootmist säilitada,free,"Kuna riik näitab toetustega üles huvi , usuvad...",79.2,79.2,87.5
2750,39414,Train/tea_AA_05_6_ud211.conllu,Kui protsessorüksus sooritab ainuvallatava ope...,141,159,süsteemi ühendatud,free,Kui protsessorüksus sooritab ainuvallatava ope...,94.7,94.7,100.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
9,36667,Train/arborest_ud211.conllu,Sõmera toetab plaani viia umbes viiendik telek...,21,86,viia umbes viiendik telekomi aktsiatest börsil...,free,Sõmera toetab plaani .,100.0,100.0,100.0
11,36669,Train/aja_epl20061216_ud211.conllu,Keset steppi asuvas Astanas langeb talviti tem...,0,19,Keset steppi asuvas,free,Astanas langeb talviti temperatuur -30 kraadini .,100.0,100.0,100.0
19,36677,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Meditsiini areng on toonud pöörde mitme varem ...,40,76,varem paratamatult fataalseks peetud,free,Meditsiini areng on toonud pöörde mitme haigus...,100.0,100.0,100.0
24,36682,Train/tea_toohoive.conllu,"Nimetatud "" ülekandemehhanismide "" uurimine pe...",0,9,Nimetatud,free,""" ülekandemehhanismide "" uurimine peaks andma ...",100.0,100.0,100.0
26,36684,Train/tea_geofyysika.conllu,Enamus teaduse ajalukku läinud vanakreeka kuul...,7,30,teaduse ajalukku läinud,free,Enamus vanakreeka kuulsusi on elanud sellel aj...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2715,39378,Train/aja_sloleht20071217_ud211.conllu,Igal juhul peaksid mängu põhjal tehtud järeldu...,19,38,mängu põhjal tehtud,free,Igal juhul peaksid järeldused sobima pärismaai...,100.0,100.0,100.0
2722,39385,Train/tea_geofyysika.conllu,Loogilise mõtlemise teel konstrueeriti maailma...,77,89,teadaolevaid,free,Loogilise mõtlemise teel konstrueeriti maailma...,100.0,100.0,100.0
2738,39402,Train/ilu_maailm_ud211.conllu,Tema tumedad juuksed ripnesid võidunud dressip...,30,38,võidunud,free,Tema tumedad juuksed ripnesid dressipluusil na...,100.0,100.0,100.0
2746,39410,Train/aja_luup200202_osa_2_ud211.conllu,Tsaariajal ja nõukogude perioodil mahavaikitud...,0,46,Tsaariajal ja nõukogude perioodil mahavaikitud,free,hõimu- ja klannisuhted elustusid NL-i lagunede...,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [14]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,36659,Train/aja_pm20001004_osa_3_ud211.conllu,Viljandi maavalitsuse haridus- ja kultuuriosak...,157,180,õppetööst eemal olevale,bound,Viljandi maavalitsuse haridus- ja kultuuriosak...,96.3,96.3,100.0
3,36661,Train/aja_pm19980926e_ud211.conllu,Uskumatult suus sulav lõpptulemus paneb õuna e...,87,127,muuta iseenesest lihtne kook taevalikuks,bound,Uskumatult suus sulav lõpptulemus paneb õuna e...,92.9,92.9,100.0
6,36664,Train/aja_luup200202_osa_3_ud211.conllu,"Kui erinevused puuduvad , polnud radioaktiivne...",126,146,riski suurendavateks,bound,"Kui erinevused puuduvad , polnud radioaktiivne...",100.0,100.0,100.0
7,36665,Train/aja_pm19980926e_ud211.conllu,Möödunud aastal viiekümnenda juubeli tähistami...,0,8,Möödunud,bound,aastal viiekümnenda juubeli tähistamiseks korr...,81.8,100.0,81.8
10,36668,Train/aja_ee200110_osa_5_ud211.conllu,""" Olen mänginud plastiliiniga sellest ajast , ...",44,67,", kui olin põlvepikkune",bound,""" Olen mänginud plastiliiniga sellest ajast , ...",100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2742,39406,Train/aja_horisont2000_osa_1_ud211.conllu,Aafrikas tehtud järeleandmistele lootes tegi C...,181,187,eitava,bound,Aafrikas tehtud järeleandmistele lootes tegi C...,44.0,48.0,48.0
2743,39407,Dev/tea_eesti_arst_2004_osa_1_ud211.conllu,Kõigis neis on vähemalt kaks kliinilist kritee...,96,161,", et dementsus peaks olema põhjustatud vaskula...",bound,Kõigis neis on vähemalt kaks kliinilist kritee...,100.0,100.0,100.0
2748,39412,Train/tea_toohoive.conllu,Nelja viimase nädala jooksul kursustel osalenu...,0,48,Nelja viimase nädala jooksul kursustel osalenute,bound,osakaal,100.0,100.0,100.0
2749,39413,Train/tea_AA_05_6_ud211.conllu,Kui protsessorüksus sooritab ainuvallatava ope...,29,42,ainuvallatava,bound,Kui protsessorüksus sooritab operandi laadimis...,90.0,90.0,95.0


In [19]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [15]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
4,36662,Train/aja_ee199920_osa_1_ud211.conllu,"Selle asemel , et avaldada vabandus oma töötaj...",13,74,", et avaldada vabandus oma töötaja ebaõnnestun...",redundant comma,"Selle asemel , asub toimetus Tanel Padarit eda...",82.8,93.1,89.7
133,36791,Train/aja_ee200110_osa_7_ud211.conllu,Nii kirjutas naistest Juhan Liivi põlvkonnakaa...,59,96,apolloonilist kultuuritüüpi esindavad,redundant comma,Nii kirjutas naistest Juhan Liivi põlvkonnakaa...,95.0,95.0,100.0
160,36818,Train/aja_ee200110_osa_7_ud211.conllu,EDUARD VILDE KIRJELDAB juba sada aastat tagasi...,126,151,korsettidesse kängitsetud,redundant comma,EDUARD VILDE KIRJELDAB juba sada aastat tagasi...,52.6,57.9,73.7
319,36977,Train/tea_toohoive.conllu,"Selleks , et hinnata keskmise palga ja miinimu...",8,67,", et hinnata keskmise palga ja miinimumpalga m...",redundant comma,"Selleks , tuleb vaadata ka seda , mis on juhtu...",92.9,92.9,100.0
848,37507,Train/ilu_kivirahk_ud211.conllu,Ott ei olnud Pierre'i ideedest küll suuremat s...,70,124,", kas mingisugune abstraktne Tõde on olemas võ...",redundant comma,Ott ei olnud Pierre'i ideedest küll suuremat s...,93.8,100.0,93.8
899,37558,Dev/tea_eesti_arst_2004_osa_1_ud211.conllu,I klass - tõendatud rohkem kui ühe randomiseer...,35,50,randomiseeritud,redundant comma,"I klass - tõendatud rohkem kui ühe , kontrollg...",61.2,67.3,79.6
911,37570,Train/aja_ee200110_osa_3_ud211.conllu,Jewitti kalkulatsioonide kohaselt on tõenäosus...,47,101,", et inimene oma elu jooksul taevakeha läbi su...",redundant comma,Jewitti kalkulatsioonide kohaselt on tõenäosus...,None,None,None
966,37625,Train/aja_horisont2000_osa_3_ud211.conllu,"Kui siirdame selleks vaid ühe geeni , siis pea...",103,117,meid huvitavat,redundant comma,"Kui siirdame selleks vaid ühe geeni , siis pea...",95.5,95.5,95.5
974,37633,Dev/tea_eesti_arst_2004_osa_1_ud211.conllu,See et meestel leidsime kromosoomiaberratsioon...,4,87,et meestel leidsime kromosoomiaberratsioone um...,redundant comma,"See , ühtib kirjanduse andmetega .",100.0,100.0,100.0
994,37653,Train/tea_toohoive.conllu,"Piltlikult öeldes , vahetasid pika- ja lühiaja...",75,169,", mil üldine tööpuudus jäi suhteliselt stabiil...",redundant comma,"Piltlikult öeldes , vahetasid pika- ja lühiaja...",95.0,95.0,100.0


In [20]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [16]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
294,36952,Train/aja_ee200110_osa_6_ud211.conllu,"Shveitslased ütlesid rahvaküsitlusel kindla "" ...",64,105,kehtestada asulates piirkiiruseks 30 km/h,unnatural,"Shveitslased ütlesid rahvaküsitlusel kindla "" ...",77.8,77.8,88.9
420,37078,Train/tea_geofyysika.conllu,Käesoleval ajal kasvab ööpäeva kestus 0.0023 s...,0,10,Käesoleval,unnatural,ajal kasvab ööpäeva kestus 0.0023 sekundit saj...,100.0,100.0,100.0
605,37264,Train/aja_pm20000218_osa_5_ud211.conllu,"Söödi üks parimaid sõpru Vootele Ustav läks , ...",54,69,tädilt laenatud,unnatural,"Söödi üks parimaid sõpru Vootele Ustav läks , ...",76.5,76.5,94.1
723,37382,Train/aja_ml200247_osa_2_ud211.conllu,"Kuivõrd sobib väikeriik Eestile see , et suurr...",36,110,", et suurriik Venemaa ja suurriik Saksamaa saa...",unnatural,Kuivõrd sobib väikeriik Eestile see ?,66.7,83.3,66.7
738,37397,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Ajendatuna patsiendi kaebustest ja anamnestili...,0,61,Ajendatuna patsiendi kaebustest ja anamnestili...,unnatural,"tehti KT-uuring kopsudest , millel oli näha va...",76.9,80.8,96.2
746,37405,Train/aja_pm20001004_osa_1_ud211.conllu,Henn Elmeti hinnangul muudab põllumajandusülik...,75,146,", et selle teaduskonnad ja instituudid on omav...",unnatural,Henn Elmeti hinnangul muudab põllumajandusülik...,90.0,90.0,100.0
803,37462,Train/aja_ee200110_osa_5_ud211.conllu,"Kõrvale lähevad need kalad , mille saba on "" n...",27,95,", mille saba on "" normist väljas "" , kas liiga...",unnatural,Kõrvale lähevad need kalad .,100.0,100.0,100.0
843,37502,Train/aja_pm20001004_osa_2_ud211.conllu,"Eesti maksusüsteem on selline , et omavalitsus...",30,111,", et omavalitsuste sissetulekud sõltuvad suure...",unnatural,Eesti maksusüsteem on selline .,80.0,100.0,80.0
850,37509,Train/aja_ee200110_osa_2_ud211.conllu,""" Ma kardan , et mälestus inimesest rikutakse ...",58,101,", et kuriteo motiive pole võimalik tõestada",unnatural,""" Ma kardan , et mälestus inimesest rikutakse ...",100.0,100.0,100.0
875,37534,Train/aja_ee200110_osa_3_ud211.conllu,"Sama hästi ei läinud dinosaurustel , kes arvat...",53,135,just 65 miljoni aasta taguse Mehhikosse Chicxc...,unnatural,"Sama hästi ei läinud dinosaurustel , kes arvat...",100.0,100.0,100.0


In [21]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [17]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
311,36969,Train/tea_AA_05_6_ud211.conllu,"Kasutaja sooritab LRE keskkonnas otsingu , pär...",73,96,konsortsiumiga liitunud,dubious,"Kasutaja sooritab LRE keskkonnas otsingu , pär...",100.0,100.0,100.0


In [22]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
675,37334,Train/ilu_kivirahk_ud211.conllu,Kitli alt - see oli ka üsna lühike ! - paistsi...,10,34,- see oli ka üsna lühike,other redundant punctuation,Kitli alt ! - paistsid hästi karvased jalad ni...,53.8,53.8,92.3
1100,37759,Train/aja_sloleht20071217_ud211.conllu,""" See õhtu näeb välja niimoodi , et kell 19 on...",90,192,", kus lisaks päevakajalistele uudistele tehaks...",other redundant punctuation,""" See õhtu näeb välja niimoodi , et kell 19 on...",47.6,47.6,71.4
2123,38786,Train/tea_AA_05_6_ud211.conllu,Pigem olgu andmekaevanduse definitsioon üldise...,50,62,nagu pakutud,other redundant punctuation,Pigem olgu andmekaevanduse definitsioon üldise...,76.0,76.0,92.0
2207,38870,Train/aja_horisont2000_osa_2_ud211.conllu,Nelinurkse põhiplaaniga puhtaveeline tiik - ne...,70,114,kus prisked vääriskalad laisalt ringi ujuvad,other redundant punctuation,Nelinurkse põhiplaaniga puhtaveeline tiik - ne...,90.9,100.0,90.9
2562,39225,Train/arborest_ud211.conllu,""" Neid kahte vaadates meenub kulunud tsitaat ""...",72,108,mitu korda ma pean üht ja sama meest,other redundant punctuation,""" Neid kahte vaadates meenub kulunud tsitaat ""...",78.6,92.9,85.7


In [23]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:16:30.267244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:16:30.603681: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:16:31.789576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:16:31.789695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data




1566it [06:39,  3.92it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")